<a href="https://colab.research.google.com/github/patriciacarvalhom/Trabalho-de-BD3/blob/bd3/naruto_rachel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
bd_name = "bd6"

!apt-get update -y
!apt-get install postgresql postgresql-contrib -y
!service postgresql start
!sudo -u postgres dropdb --if-exists "$bd_name"
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres createdb "$bd_name"

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,842 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,848 kB in 2s (1,599 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository '

In [ ]:
!pip install psycopg2-binary

import psycopg2
import pandas as pd

conn = psycopg2.connect(
    dbname=bd_name,
    user="postgres",
    password="postgres",
    host="localhost"
)

cur = conn.cursor()
query = "SELECT * FROM Cla;"
cur.execute(query)
df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
print(df)
# df.to_csv("Cla.csv", index=False)
# print("CSV gerado com sucesso!")
cur.close()
conn.close()

  id_cla  nome_cla
0     C1    Uchiha
1     C2     Hyuga
2     C3     Senju
3     C4      Nara
4     C5  Akimichi
5     C6  Yamanaka
6     C7   Inuzuka
7     C8   Aburame
8     C9  Sarutobi
9    C10     Senju


In [ ]:
script_sql = """

-- -----------------------------------------------------
-- Tabela Clã
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Cla (
    id_cla VARCHAR(5) PRIMARY KEY,
    nome_cla VARCHAR(100)
);

-- -----------------------------------------------------
-- Tabela Vila
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Vila (
    id_vila VARCHAR(5) PRIMARY KEY,
    nome_vila VARCHAR(100)
);

-- -----------------------------------------------------
-- Tabela Missao
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Missao (
    id_missao VARCHAR(5) PRIMARY KEY,
    descricao TEXT,
    ranking CHAR(1)
);

-- -----------------------------------------------------
-- Tabela Time
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Time (
    id_time VARCHAR(5) PRIMARY KEY,
    nome_time VARCHAR(100)
);

-- -----------------------------------------------------
-- Tabela Invocacao
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Invocacao (
    id_invocacao VARCHAR(5) PRIMARY KEY,
    nome_invocacao VARCHAR(100)
);

-- -----------------------------------------------------
-- Tabela Chakra
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Chakra (
    id_chakra VARCHAR(5) PRIMARY KEY,
    natureza VARCHAR(100),
    cor VARCHAR(50),
    rotacao VARCHAR(50)
);

-- -----------------------------------------------------
-- Tabela Jutsu
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Jutsu (
    id_jutsu VARCHAR(5) PRIMARY KEY,
    nome_jutsu VARCHAR(100),
    tipo_jutsu VARCHAR(50)
);

-- -----------------------------------------------------
-- Tabela Arma
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Arma (
    id_arma VARCHAR(5) PRIMARY KEY,
    nome_arma VARCHAR(100),
    tipo_arma VARCHAR(50)
);

-- -----------------------------------------------------
-- Tabela Selo
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Selo (
    id_selo VARCHAR(5) PRIMARY KEY,
    nome_selo VARCHAR(100)
);

-- -----------------------------------------------------
-- Tabela KekkeiGenkai (agora pode referenciar Cla)
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS KekkeiGenkai (
    id_kekkei VARCHAR(5) PRIMARY KEY,
    nome_kekkei VARCHAR(100),
    id_cla VARCHAR(5),
    FOREIGN KEY (id_cla) REFERENCES Cla(id_cla)
);

-- -----------------------------------------------------
-- Tabela Ninja (agora pode referenciar Vila e Cla)
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Ninja (
    id_ninja VARCHAR(5) PRIMARY KEY,
    nome_ninja VARCHAR(100),
    idade INT,
    ranking CHAR(1),
    tipo CHAR(1),
    id_vila VARCHAR(5),
    id_cla VARCHAR(5),
    FOREIGN KEY (id_vila) REFERENCES Vila(id_vila),
    FOREIGN KEY (id_cla) REFERENCES Cla(id_cla)
);

-- -----------------------------------------------------
-- Tabela Sensei (agora pode referenciar Time)
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Sensei (
    id_ninja VARCHAR(5) PRIMARY KEY,
    id_time VARCHAR(5),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_time) REFERENCES Time(id_time)
);
-- -----------------------------------------------------
-- Demais tabelas com dependências
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Bijuu (
    id_bijuu VARCHAR(5) PRIMARY KEY,
    id_ninja VARCHAR(5),
    nome_bijuu VARCHAR(100),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja)
);

CREATE TABLE IF NOT EXISTS Hokage (
    id_ninja VARCHAR(5) PRIMARY KEY,
    id_vila VARCHAR(5),
    FOREIGN KEY (id_vila) REFERENCES Vila(id_vila),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja)
);

CREATE TABLE IF NOT EXISTS Mestres (
    id_aprendiz VARCHAR(5),
    id_mestre VARCHAR(5) PRIMARY KEY,
    especialidade VARCHAR(100),
    FOREIGN KEY (id_aprendiz) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_mestre) REFERENCES Ninja(id_ninja)
);

-- Tabelas de relacionamento
CREATE TABLE IF NOT EXISTS Ninja_Arma (
    id_ninja VARCHAR(5),
    id_arma VARCHAR(5),
    PRIMARY KEY (id_ninja, id_arma),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_arma) REFERENCES Arma(id_arma)
);

CREATE TABLE IF NOT EXISTS Ninja_Invocacao (
    id_ninja VARCHAR(5),
    id_invocacao VARCHAR(5),
    PRIMARY KEY (id_ninja, id_invocacao),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_invocacao) REFERENCES Invocacao(id_invocacao)
);

CREATE TABLE IF NOT EXISTS Ninja_Chakra (
    id_ninja VARCHAR(5),
    id_chakra VARCHAR(5),
    PRIMARY KEY (id_ninja, id_chakra),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_chakra) REFERENCES Chakra(id_chakra)
);

CREATE TABLE IF NOT EXISTS Jutsu_Chakra (
    id_jutsu VARCHAR(5),
    id_chakra VARCHAR(5),
    PRIMARY KEY (id_jutsu, id_chakra),
    FOREIGN KEY (id_jutsu) REFERENCES Jutsu(id_jutsu),
    FOREIGN KEY (id_chakra) REFERENCES Chakra(id_chakra)
);

CREATE TABLE IF NOT EXISTS Selo_Jutsu (
    id_selo VARCHAR(5),
    id_jutsu VARCHAR(5),
    PRIMARY KEY (id_selo, id_jutsu),
    FOREIGN KEY (id_selo) REFERENCES Selo(id_selo),
    FOREIGN KEY (id_jutsu) REFERENCES Jutsu(id_jutsu)
);

CREATE TABLE IF NOT EXISTS Ninja_Time (
    id_ninja VARCHAR(5),
    id_time VARCHAR(5),
    PRIMARY KEY (id_ninja, id_time),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_time) REFERENCES Time(id_time)
);

CREATE TABLE IF NOT EXISTS Time_Missao (
    id_missao VARCHAR(5),
    id_time VARCHAR(5),
    PRIMARY KEY (id_missao, id_time),
    FOREIGN KEY (id_missao) REFERENCES Missao(id_missao),
    FOREIGN KEY (id_time) REFERENCES Time(id_time)
);

CREATE TABLE IF NOT EXISTS Ninja_Kekkei (
    id_ninja VARCHAR(5),
    id_kekkei VARCHAR(5),
    PRIMARY KEY (id_ninja, id_kekkei),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_kekkei) REFERENCES KekkeiGenkai(id_kekkei)
);

-- Inserção de dados

INSERT INTO Cla (id_cla, nome_cla) VALUES
('C1', 'Uchiha'),
('C2', 'Hyuga'),
('C3', 'Senju'),
('C4', 'Nara'),
('C5', 'Akimichi'),
('C6', 'Yamanaka'),
('C7', 'Inuzuka'),
('C8', 'Aburame'),
('C9', 'Sarutobi'),
('C10', 'Senju')
ON CONFLICT (id_cla) DO NOTHING;

INSERT INTO Time (id_time, nome_time) VALUES
('T001', 'Time Kakashi'),
('T002', 'Time Asuma'),
('T003', 'Time Kurenai'),
('T004', 'Time Gai'),
('T005', 'Time Minato'),
('T006', 'Time Gaara'),
('T007', 'Time Temari'),
('T008', 'Time Kankuro'),
('T009', 'Time Baki'),
('T010', 'Time da Areia Elite'),
('T011', 'Time Mei Terumi'),
('T012', 'Time Chojuro'),
('T013', 'Time Zabuza'),
('T014', 'Time Espadachins da Névoa'),
('T015', 'Time da Névoa Elite')
ON CONFLICT (id_time) DO NOTHING;

INSERT INTO Vila(id_vila, nome_vila) VALUES
('001', 'Vila da Folha'),
('002', 'Vila da Areia'),
('003', 'Vila da Névoa')
ON CONFLICT (id_vila) DO NOTHING;

INSERT INTO Missao (id_missao, descricao, ranking) VALUES
  ('M001','Proteger o Daimyo em viagem oficial','S'),
  ('M002','Neutralizar organização criminosa internacional','S'),
  ('M003','Capturar ninja renegado de alto nível','S'),
  ('M004','Resgatar reféns em território inimigo','S'),
  ('M005','Eliminar ameaça classe S nas fronteiras','S'),
  ('M006','Destruir base secreta de mercenários','S'),
  ('M007','Escoltar jinchuuriki sob ataque','S'),
  ('M008','Infiltrar-se em vila hostil e coletar informações críticas','S'),
  ('M009','Conter invasão em larga escala','S'),
  ('M010','Investigar desaparecimento de equipe ANBU','S'),

  -- Missões rank A
  ('M011','Capturar bandido classe A nas montanhas','A'),
  ('M012','Proteger caravana de comerciantes internacionais','A'),
  ('M013','Destruir laboratório ilegal de armas ninjas','A'),
  ('M014','Escoltar pesquisador importante até a Vila da Folha','A'),
  ('M015','Interceptar mensageiro inimigo com informações sigilosas','A'),

  -- Missões rank B
  ('M016','Investigar sabotagem em depósitos de suprimentos','B'),
  ('M017','Patrulhar estrada principal entre vilas aliadas','B'),
  ('M018','Capturar grupo de ladrões de pergaminhos','B'),
  ('M019','Proteger engenheiros reconstruindo ponte estratégica','B'),
  ('M020','Resgatar viajantes desaparecidos em floresta densa','B'),

  -- Missões rank C
  ('M021','Escoltar mercadorias pela estrada rural','C'),
  ('M022','Ajudar agricultores durante a colheita','C'),
  ('M023','Buscar ervas medicinais em área distante','C'),
  ('M024','Acompanhar professor em excursão escolar','C'),
  ('M025','Proteger pequeno templo em vila vizinha','C'),

  -- Missões rank D
  ('M026','Entregar documentos simples entre prédios da vila','D'),
  ('M027','Ajudar a limpar praça após festival','D'),
  ('M028','Levar mantimentos ao posto de guarda','D'),
  ('M029','Acompanhar criança até a escola da vila','D'),
  ('M030','Recolher materiais no armazém comunitário','D')
ON CONFLICT (id_missao) DO NOTHING;

INSERT INTO Ninja (id_ninja, nome_ninja, idade, ranking, tipo, id_vila, id_cla) VALUES
    -- Vila da Folha (001) - Clãs clássicos da Folha
    ('N001', 'Itachi Uchiha', 21, 'S', 'J', '001', 'C1'),
    ('N002', 'Sasuke Uchiha', 16, 'A', 'C', '001', 'C1'),
    ('N003', 'Shisui Uchiha', 19, 'S', 'J', '001', 'C1'),
    ('N004', 'Fugaku Uchiha', 40, 'A', 'J', '001', 'C1'),

    ('N005', 'Hinata Hyuga', 16, 'A', 'G', '001', 'C2'),
    ('N006', 'Neji Hyuga', 17, 'A', 'J', '001', 'C2'),
    ('N007', 'Hiashi Hyuga', 42, 'S', 'J', '001', 'C2'),

    ('N008', 'Hashirama Senju', 35, 'S', 'J', '001', 'C3'),
    ('N009', 'Tobirama Senju', 34, 'S', 'J', '001', 'C3'),
    ('N010', 'Tsunade Senju', 36, 'S', 'J', '001', 'C3'),

    ('N011', 'Shikamaru Nara', 16, 'A', 'C', '001', 'C4'),
    ('N012', 'Shikaku Nara', 32, 'S', 'J', '001', 'C4'),

    ('N013', 'Choji Akimichi', 16, 'C', 'G', '001', 'C5'),
    ('N014', 'Choza Akimichi', 34, 'C', 'J', '001', 'C5'),

    ('N015', 'Ino Yamanaka', 16, 'B', 'G', '001', 'C6'),
    ('N016', 'Inoichi Yamanaka', 35, 'A', 'J', '001', 'C6'),

    ('N017', 'Kiba Inuzuka', 16, 'B', 'G', '001', 'C7'),
    ('N018', 'Tsume Inuzuka', 33, 'A', 'J', '001', 'C7'),

    ('N019', 'Shino Aburame', 16, 'A', 'C', '001', 'C8'),
    ('N020', 'Shibi Aburame', 35, 'A', 'J', '001', 'C8'),

    ('N021', 'Hiruzen Sarutobi', 68, 'S', 'J', '001', 'C9'),
    ('N022', 'Asuma Sarutobi', 31, 'A', 'J', '001', 'C9'),
    ('N023', 'Konohamaru Sarutobi', 12, 'C', 'G', '001', 'C9'),

    ('N024', 'Nawaki Senju', 12, 'B', 'G', '001', 'C10'),
    ('N025', 'Arata Senju', 20, 'B', 'C', '001', 'C10'),

    -- Vila da Areia (002) - usando os mesmos 10 clãs já cadastrados
    ('N026', 'Gaara', 16, 'S', 'J', '002', 'C3'),
    ('N027', 'Temari', 17, 'A', 'J', '002', 'C4'),
    ('N028', 'Kankuro', 17, 'A', 'J', '002', 'C5'),
    ('N029', 'Baki', 35, 'A', 'J', '002', 'C4'),
    ('N030', 'Yashamaru', 30, 'B', 'C', '002', 'C6'),
    ('N031', 'Maki', 18, 'B', 'G', '002', 'C6'),
    ('N032', 'Shira', 20, 'A', 'C', '002', 'C5'),
    ('N033', 'Suna Shinobi A', 19, 'C', 'G', '002', 'C7'),
    ('N034', 'Suna Shinobi B', 21, 'C', 'C', '002', 'C8'),
    ('N035', 'Suna Médico', 23, 'B', 'G', '002', 'C8'),

    ('N036', 'Mei Terumi', 30, 'S', 'J', '003', 'C2'),
    ('N037', 'Zabuza Momochi', 26, 'A', 'J', '003', 'C1'),
    ('N038', 'Haku', 15, 'B', 'G', '003', 'C1'),
    ('N039', 'Chojuro', 20, 'A', 'C', '003', 'C4'),
    ('N040', 'Ao', 28, 'A', 'J', '003', 'C6'),
    ('N041', 'Suigetsu Hozuki', 18, 'B', 'C', '003', 'C3'),
    ('N042', 'Mangetsu Hozuki', 20, 'A', 'J', '003', 'C3'),
    ('N043', 'Kisame Hoshigaki', 32, 'S', 'J', '003', 'C7'),
    ('N050', 'Kakashi Hatake', 32, 'S', 'J', '001', 'C3'),          -- Líder do Time Kakashi
    ('N051', 'Asuma Sarutobi', 31, 'A', 'J', '001', 'C9'),          -- Líder do Time Asuma
    ('N052', 'Kurenai Yuhi', 30, 'A', 'J', '001', 'C6'),            -- Líder do Time Kurenai
    ('N053', 'Might Gai', 32, 'S', 'J', '001', 'C5'),               -- Líder do Time Gai
    ('N054', 'Minato Namikaze', 35, 'S', 'J', '001', 'C3');         -- Líder do Time Minato
    ('N055', 'Gaara Sabaku', 20, 'S', 'J', '002', 'C3'),            -- Time Gaara
    ('N056', 'Temari Sabaku', 19, 'A', 'J', '002', 'C4'),           -- Time Temari
    ('N057', 'Kankuro Sabaku', 19, 'A', 'J', '002', 'C5'),          -- Time Kankuro
    ('N058', 'Baki Sabaku', 40, 'A', 'J', '002', 'C4'),             -- Time Baki
    ('N059', 'Capitão da Areia Elite', 33, 'S', 'J', '002', 'C8');  -- Time da Areia Elite
    ('N060', 'Mei Terumi', 30, 'S', 'J', '003', 'C2'),              -- Time Mei Terumi
    ('N061', 'Chojuro Hozuki', 22, 'A', 'J', '003', 'C3'),          -- Time Chojuro
    ('N062', 'Zabuza Momochi', 28, 'A', 'J', '003', 'C1'),          -- Time Zabuza
    ('N063', 'Líder dos Espadachins', 35, 'S', 'J', '003', 'C7'),   -- Time Espadachins
    ('N064', 'Capitão da Névoa Elite', 34, 'S', 'J', '003', 'C6')   -- Time da Névoa Elite
ON CONFLICT (id_ninja) DO NOTHING;

INSERT INTO Time_Missao (id_missao, id_time) VALUES
    -- T001 - mais missÃµes e mais S
    ('M001','T001'),
    ('M002','T001'),
    ('M011','T001'),
    ('M016','T001'),
    ('M017','T001'),
    ('M021','T001'),
    ('M026','T001'),

    -- T002
    ('M003','T002'),
    ('M004','T002'),
    ('M012','T002'),
    ('M018','T002'),
    ('M022','T002'),
    ('M027','T002'),

    -- T003
    ('M005','T003'),
    ('M006','T003'),
    ('M013','T003'),
    ('M019','T003'),
    ('M023','T003'),
    ('M028','T003'),

    -- T004
    ('M007','T004'),
    ('M014','T004'),
    ('M016','T004'),
    ('M020','T004'),
    ('M024','T004'),
    ('M026','T004'),
    ('M029','T004'),

    -- T005
    ('M008','T005'),
    ('M015','T005'),
    ('M017','T005'),
    ('M021','T005'),
    ('M025','T005'),
    ('M030','T005'),

    -- T006
    ('M009','T006'),
    ('M011','T006'),
    ('M018','T006'),
    ('M022','T006'),
    ('M026','T006'),

    -- T007
    ('M010','T007'),
    ('M012','T007'),
    ('M019','T007'),
    ('M023','T007'),
    ('M027','T007'),

    -- T008
    ('M001','T008'),
    ('M013','T008'),
    ('M020','T008'),
    ('M021','T008'),
    ('M028','T008'),

    -- T009
    ('M002','T009'),
    ('M014','T009'),
    ('M016','T009'),
    ('M022','T009'),
    ('M029','T009'),

    -- T010
    ('M003','T010'),
    ('M015','T010'),
    ('M017','T010'),
    ('M023','T010'),
    ('M030','T010'),

    -- T011
    ('M004','T011'),
    ('M011','T011'),
    ('M018','T011'),
    ('M024','T011'),
    ('M027','T011'),

    -- T012
    ('M005','T012'),
    ('M012','T012'),
    ('M019','T012'),
    ('M025','T012'),
    ('M026','T012'),

    -- T013
    ('M006','T013'),
    ('M013','T013'),
    ('M020','T013'),
    ('M021','T013'),
    ('M027','T013'),

    -- T014
    ('M007','T014'),
    ('M014','T014'),
    ('M017','T014'),
    ('M022','T014'),
    ('M028','T014'),

    -- T015
    ('M008','T015'),
    ('M015','T015'),
    ('M018','T015'),
    ('M023','T015'),
    ('M029','T015')
ON CONFLICT (id_missao, id_time) DO NOTHING;

INSERT INTO Ninja_Time (id_ninja, id_time) VALUES
    -- Vila da Folha (001) -> Times T001 a T005

    -- T001 - Time Kakashi
    ('N001', 'T001'),  -- Itachi Uchiha (S)
    ('N002', 'T001'),  -- Sasuke Uchiha (A)
    ('N011', 'T001'),  -- Shikamaru Nara (A)
    ('N013', 'T001'),  -- Choji Akimichi (C)
    ('N015', 'T001'),  -- Ino Yamanaka (B)

    -- T002 - Time Asuma
    ('N003', 'T002'),  -- Shisui Uchiha (S)
    ('N004', 'T002'),  -- Fugaku Uchiha (A)
    ('N012', 'T002'),  -- Shikaku Nara (S)
    ('N014', 'T002'),  -- Choza Akimichi (C)
    ('N017', 'T002'),  -- Kiba Inuzuka (B)

    -- T003 - Time Kurenai
    ('N005', 'T003'),  -- Hinata Hyuga (A)
    ('N006', 'T003'),  -- Neji Hyuga (A)
    ('N007', 'T003'),  -- Hiashi Hyuga (S)
    ('N023', 'T003'),  -- Konohamaru Sarutobi (C)
    ('N024', 'T003'),  -- Nawaki Senju (B)

    -- T004 - Time Gai
    ('N008', 'T004'),  -- Hashirama Senju (S)
    ('N009', 'T004'),  -- Tobirama Senju (S)
    ('N018', 'T004'),  -- Tsume Inuzuka (A)
    ('N020', 'T004'),  -- Shibi Aburame (A)
    ('N025', 'T004'),  -- Arata Senju (B)

    -- T005 - Time Minato
    ('N010', 'T005'),  -- Tsunade Senju (S)
    ('N016', 'T005'),  -- Inoichi Yamanaka (A)
    ('N019', 'T005'),  -- Shino Aburame (A)
    ('N021', 'T005'),  -- Hiruzen Sarutobi (S)
    ('N022', 'T005'),  -- Asuma Sarutobi (A)

    -- Vila da Areia (002) -> Times T006 a T010

    -- T006 - Time Gaara
    ('N026', 'T006'),  -- Gaara (S)
    ('N027', 'T006'),  -- Temari (A)

    -- T007 - Time Temari
    ('N028', 'T007'),  -- Kankuro (A)
    ('N029', 'T007'),  -- Baki (A)

    -- T008 - Time Kankuro
    ('N030', 'T008'),  -- Yashamaru (B)
    ('N032', 'T008'),  -- Shira (A)

    -- T009 - Time Baki
    ('N031', 'T009'),  -- Maki (B)
    ('N033', 'T009'),  -- Suna Shinobi A (C)

    -- T010 - Time da Areia Elite
    ('N034', 'T010'),  -- Suna Shinobi B (C)
    ('N035', 'T010'),  -- Suna Médico (B)

    -- Vila da Névoa (003) -> Times T011 a T015

    -- T011 - Time Mei Terumi
    ('N036', 'T011'),  -- Mei Terumi (S)
    ('N039', 'T011'),  -- Chojuro (A)

    -- T012 - Time Chojuro
    ('N037', 'T012'),  -- Zabuza Momochi (A)
    ('N038', 'T012'),  -- Haku (B)

    -- T013 - Time Zabuza
    ('N040', 'T013'),  -- Ao (A)
    ('N041', 'T013'),  -- Suigetsu Hozuki (B)

    -- T014 - Time zabuza
    ('N042', 'T014'),  -- Mangetsu Hozuki (A)

    -- T015 - Time haku
    ('N043', 'T015')   -- Kisame Hoshigaki (S)
ON CONFLICT (id_ninja, id_time) DO NOTHING;

INSERT INTO Sensei (id_ninja, id_time) VALUES
    -- Vila da Folha (001)
    ('N050', 'T001'),  -- Kakashi Hatake      -> Time Kakashi
    ('N051', 'T002'),  -- Asuma Sarutobi      -> Time Asuma
    ('N052', 'T003'),  -- Kurenai Yuhi        -> Time Kurenai
    ('N053', 'T004'),  -- Might Gai           -> Time Gai
    ('N054', 'T005'),  -- Minato Namikaze     -> Time Minato

    -- Vila da Areia (002)
    ('N055', 'T006'),  -- Gaara Sabaku              -> Time Gaara
    ('N056', 'T007'),  -- Temari Sabaku             -> Time Temari
    ('N057', 'T008'),  -- Kankuro Sabaku            -> Time Kankuro
    ('N058', 'T009'),  -- Baki Sabaku               -> Time Baki
    ('N059', 'T010'),  -- Capitão da Areia Elite    -> Time da Areia Elite

    -- Vila da Névoa (003)
    ('N060', 'T011'),  -- Mei Terumi                -> Time Mei Terumi
    ('N061', 'T012'),  -- Chojuro Hozuki            -> Time Chojuro
    ('N062', 'T013'),  -- Zabuza Momochi            -> Time Zabuza
    ('N063', 'T014'),  -- Líder dos Espadachins     -> Time Espadachins da Névoa
    ('N064', 'T015')   -- Capitão da Névoa Elite    -> Time da Névoa Elite
ON CONFLICT (id_sensei) DO NOTHING;

"""
with open("script.sql", "w") as f:
    f.write(script_sql)

!sudo -u postgres psql -d "$bd_name" -f script.sql

psql:script.sql:9: NOTICE:  relation "cla" already exists, skipping
CREATE TABLE
psql:script.sql:17: NOTICE:  relation "vila" already exists, skipping
CREATE TABLE
psql:script.sql:26: NOTICE:  relation "missao" already exists, skipping
CREATE TABLE
psql:script.sql:34: NOTICE:  relation "time" already exists, skipping
CREATE TABLE
psql:script.sql:42: NOTICE:  relation "invocacao" already exists, skipping
CREATE TABLE
psql:script.sql:52: NOTICE:  relation "chakra" already exists, skipping
CREATE TABLE
psql:script.sql:61: NOTICE:  relation "jutsu" already exists, skipping
CREATE TABLE
psql:script.sql:70: NOTICE:  relation "arma" already exists, skipping
CREATE TABLE
psql:script.sql:78: NOTICE:  relation "selo" already exists, skipping
CREATE TABLE
psql:script.sql:88: NOTICE:  relation "kekkeigenkai" already exists, skipping
CREATE TABLE
psql:script.sql:103: NOTICE:  relation "ninja" already exists, skipping
CREATE TABLE
psql:script.sql:113: NOTICE:  relation "sensei" already exists, skipp

In [ ]:
conn = psycopg2.connect(
    dbname=bd_name,
    user="postgres",
    password="postgres",
    host="localhost"
)

cur = conn.cursor()

# SELECT para verificar
query_select = "SELECT * FROM Vila;"
cur.execute(query_select)
df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
print(df)
# df.to_csv("vila.csv", index=False)
# print("CSV gerado com sucesso!")

cur.close()
conn.close()

  id_vila      nome_vila
0     001  Vila da Folha
1     002  Vila da Areia
2     003  Vila da Névoa


In [ ]:
# conn = psycopg2.connect(
#     dbname=bd_name,
#     user="postgres",
#     password="postgres",
#     host="localhost"
# )

# cur = conn.cursor()

# # 1. INSERT
# query_insert = """INSERT INTO Ninja (id_ninja, nome, idade, ranking,tipo, id_vila, id_cla) VALUES
# ('001', 'Itachi Uchiha', 18, 'S', 'J', '001', '001'),
# ('002', 'Mei Terumi', 31, 'S','J' , '003', '002'),
# ('003', 'Gaara', 18, 'S','J' , '002', '003');"""
# cur.execute(query_insert)
# conn.commit()  # ← Salva os dados!
# print("ninjas inseridas com sucesso!")

# # 2. SELECT para verificar (agora sim, fetchall())
# query_select = "SELECT * FROM Ninja;"
# cur.execute(query_select)
# df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
# print(df)
# df.to_csv("ninja.csv", index=False)
# print("CSV gerado com sucesso!")

# cur.close()
# conn.close()

In [ ]:
conn = psycopg2.connect(
    dbname=bd_name,
    user="postgres",
    password="postgres",
    host="localhost"
)

cur = conn.cursor()

query = """
SELECT
    Ninja.id_ninja,
    Vila.id_vila,
    SUM(CASE WHEN Missao.ranking = 'S' THEN 1 ELSE 0 END) AS missoes_rank_s,
    ROUND(
        CAST(SUM(CASE WHEN Missao.ranking = 'S' THEN 1 ELSE 0 END) AS NUMERIC) /
        NULLIF(COUNT(tm.id_missao), 0) * 100, 2
    ) AS porcentagem_rank_s,
    tm.id_time,
    tm.id_missao
FROM Ninja
INNER JOIN Vila ON Ninja.id_vila = Vila.id_vila
INNER JOIN Ninja_Time nt ON Ninja.id_ninja = nt.id_ninja
INNER JOIN Time_Missao tm ON nt.id_time = tm.id_time
INNER JOIN Missao ON tm.id_missao = Missao.id_missao
GROUP BY Ninja.id_ninja, Vila.id_vila, tm.id_time, tm.id_missao
HAVING SUM(CASE WHEN Missao.ranking = 'S' THEN 1 ELSE 0 END) > 0
ORDER BY Ninja.id_ninja;
"""
cur.execute(query)
df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
print(df)
df.to_csv("teste.csv")

   id_ninja id_vila  missoes_rank_s porcentagem_rank_s id_time id_missao
0      N001     001               1             100.00    T001      M001
1      N001     001               1             100.00    T001      M002
2      N002     001               1             100.00    T001      M001
3      N002     001               1             100.00    T001      M002
4      N003     001               1             100.00    T002      M001
..      ...     ...             ...                ...     ...       ...
89     N041     003               1             100.00    T013      M001
90     N042     003               1             100.00    T014      M007
91     N042     003               1             100.00    T014      M001
92     N043     003               1             100.00    T015      M008
93     N043     003               1             100.00    T015      M001

[94 rows x 6 columns]


**Estrutura do data warehouse:**

liste, para cada ninja,quantas missões ele participou, quantas dessas são de rank S e qual a porcentagem de missões S sobre o total de missões daquele ninja

Dimensão time_missão(**id_time**, **id_missao,**  ranking, nome_time)

Dimensão ninja(**id_ninja**, **id_vila,** nome_ninja)

Tabela fato missões ninja(**id_ninja**, **id_vila**, **id_missao,** **id_time**, missoes_s, porcentagem)


In [37]:
# ==============================================================
# Instalar dependências e conectar ao Neo4j Aura
# ==============================================================

!pip install py2neo pandas --quiet

from py2neo import Graph, Node, Relationship
import pandas as pd

# 🔗 Substitua pelas suas credenciais do AuraDB
graph = Graph("neo4j+s://59c60309.databases.neo4j.io",
              auth=("neo4j", "YdSY0Y-PvR_DB1x_MOdJFrt1oXSxrF5Q1pE5ReG67zI"))

print(graph.run("RETURN 'Conectado com sucesso!' AS status").data()[0]['status'])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 6.5 MB/s eta 0:00:00
Conectado com sucesso!


In [38]:
# ==============================================================
# Criar nós e relacionamentos iniciais
# ==============================================================

# Limpa o banco
graph.run("MATCH (n) DETACH DELETE n")

# Cria nós e relacionamentos

M001_T001 = Node("Missao-Time", id_missao="M001", id_time="T001", ranking="S", nome_time="Time Kakashi")
M002_T001 = Node("Missao-Time", id_missao="M002", id_time="T001", ranking="S", nome_time="Time Kakashi")
M011_T001 = Node("Missao-Time", id_missao="M011", id_time="T001", ranking="A", nome_time="Time Kakashi")
M016_T001 = Node("Missao-Time", id_missao="M016", id_time="T001", ranking="B", nome_time="Time Kakashi")
M017_T001 = Node("Missao-Time", id_missao="M017", id_time="T001", ranking="B", nome_time="Time Kakashi")
M021_T001 = Node("Missao-Time", id_missao="M021", id_time="T001", ranking="C", nome_time="Time Kakashi")
M026_T001 = Node("Missao-Time", id_missao="M026", id_time="T001", ranking="D", nome_time="Time Kakashi")

M003_T002 = Node("Missao-Time", id_missao="M003", id_time="T002", ranking="S", nome_time="Time Asuma")
M004_T002 = Node("Missao-Time", id_missao="M004", id_time="T002", ranking="S", nome_time="Time Asuma")
M012_T002 = Node("Missao-Time", id_missao="M012", id_time="T002", ranking="A", nome_time="Time Asuma")
M018_T002 = Node("Missao-Time", id_missao="M018", id_time="T002", ranking="B", nome_time="Time Asuma")
M022_T002 = Node("Missao-Time", id_missao="M022", id_time="T002", ranking="C", nome_time="Time Asuma")
M027_T002 = Node("Missao-Time", id_missao="M027", id_time="T002", ranking="D", nome_time="Time Asuma")

M005_T003 = Node("Missao-Time", id_missao="M005", id_time="T003", ranking="S", nome_time="Time Kurenai")
M006_T003 = Node("Missao-Time", id_missao="M006", id_time="T003", ranking="S", nome_time="Time Kurenai")
M013_T003 = Node("Missao-Time", id_missao="M013", id_time="T003", ranking="A", nome_time="Time Kurenai")
M019_T003 = Node("Missao-Time", id_missao="M019", id_time="T003", ranking="B", nome_time="Time Kurenai")
M023_T003 = Node("Missao-Time", id_missao="M023", id_time="T003", ranking="C", nome_time="Time Kurenai")
M028_T003 = Node("Missao-Time", id_missao="M028", id_time="T003", ranking="D", nome_time="Time Kurenai")

M007_T004 = Node("Missao-Time", id_missao="M007", id_time="T004", ranking="S", nome_time="Time Gai")
M014_T004 = Node("Missao-Time", id_missao="M014", id_time="T004", ranking="A", nome_time="Time Gai")
M016_T004 = Node("Missao-Time", id_missao="M016", id_time="T004", ranking="B", nome_time="Time Gai")
M020_T004 = Node("Missao-Time", id_missao="M020", id_time="T004", ranking="B", nome_time="Time Gai")
M024_T004 = Node("Missao-Time", id_missao="M024", id_time="T004", ranking="C", nome_time="Time Gai")
M026_T004 = Node("Missao-Time", id_missao="M026", id_time="T004", ranking="D", nome_time="Time Gai")
M029_T004 = Node("Missao-Time", id_missao="M029", id_time="T004", ranking="D", nome_time="Time Gai")

M008_T005 = Node("Missao-Time", id_missao="M008", id_time="T005", ranking="S", nome_time="Time Minato")
M015_T005 = Node("Missao-Time", id_missao="M015", id_time="T005", ranking="A", nome_time="Time Minato")
M017_T005 = Node("Missao-Time", id_missao="M017", id_time="T005", ranking="B", nome_time="Time Minato")
M021_T005 = Node("Missao-Time", id_missao="M021", id_time="T005", ranking="C", nome_time="Time Minato")
M025_T005 = Node("Missao-Time", id_missao="M025", id_time="T005", ranking="C", nome_time="Time Minato")
M030_T005 = Node("Missao-Time", id_missao="M030", id_time="T005", ranking="D", nome_time="Time Minato")

M009_T006 = Node("Missao-Time", id_missao="M009", id_time="T006", ranking="S", nome_time="Time Gaara")
M011_T006 = Node("Missao-Time", id_missao="M011", id_time="T006", ranking="A", nome_time="Time Gaara")
M018_T006 = Node("Missao-Time", id_missao="M018", id_time="T006", ranking="B", nome_time="Time Gaara")
M022_T006 = Node("Missao-Time", id_missao="M022", id_time="T006", ranking="C", nome_time="Time Gaara")
M026_T006 = Node("Missao-Time", id_missao="M026", id_time="T006", ranking="D", nome_time="Time Gaara")

M010_T007 = Node("Missao-Time", id_missao="M010", id_time="T007", ranking="S", nome_time="Time Temari")
M012_T007 = Node("Missao-Time", id_missao="M012", id_time="T007", ranking="A", nome_time="Time Temari")
M019_T007 = Node("Missao-Time", id_missao="M019", id_time="T007", ranking="B", nome_time="Time Temari")
M023_T007 = Node("Missao-Time", id_missao="M023", id_time="T007", ranking="C", nome_time="Time Temari")
M027_T007 = Node("Missao-Time", id_missao="M027", id_time="T007", ranking="D", nome_time="Time Temari")

M001_T008 = Node("Missao-Time", id_missao="M001", id_time="T008", ranking="S", nome_time="Time Kankuro")
M013_T008 = Node("Missao-Time", id_missao="M013", id_time="T008", ranking="A", nome_time="Time Kankuro")
M020_T008 = Node("Missao-Time", id_missao="M020", id_time="T008", ranking="B", nome_time="Time Kankuro")
M021_T008 = Node("Missao-Time", id_missao="M021", id_time="T008", ranking="C", nome_time="Time Kankuro")
M028_T008 = Node("Missao-Time", id_missao="M028", id_time="T008", ranking="D", nome_time="Time Kankuro")

M002_T009 = Node("Missao-Time", id_missao="M002", id_time="T009", ranking="S", nome_time="Time Baki")
M014_T009 = Node("Missao-Time", id_missao="M014", id_time="T009", ranking="A", nome_time="Time Baki")
M016_T009 = Node("Missao-Time", id_missao="M016", id_time="T009", ranking="B", nome_time="Time Baki")
M022_T009 = Node("Missao-Time", id_missao="M022", id_time="T009", ranking="C", nome_time="Time Baki")
M029_T009 = Node("Missao-Time", id_missao="M029", id_time="T009", ranking="D", nome_time="Time Baki")

M003_T010 = Node("Missao-Time", id_missao="M003", id_time="T010", ranking="S", nome_time="Time da Areia Elite")
M015_T010 = Node("Missao-Time", id_missao="M015", id_time="T010", ranking="A", nome_time="Time da Areia Elite")
M017_T010 = Node("Missao-Time", id_missao="M017", id_time="T010", ranking="B", nome_time="Time da Areia Elite")
M023_T010 = Node("Missao-Time", id_missao="M023", id_time="T010", ranking="C", nome_time="Time da Areia Elite")
M030_T010 = Node("Missao-Time", id_missao="M030", id_time="T010", ranking="D", nome_time="Time da Areia Elite")

M004_T011 = Node("Missao-Time", id_missao="M004", id_time="T011", ranking="S", nome_time="Time Mei Terumi")
M011_T011 = Node("Missao-Time", id_missao="M011", id_time="T011", ranking="A", nome_time="Time Mei Terumi")
M018_T011 = Node("Missao-Time", id_missao="M018", id_time="T011", ranking="B", nome_time="Time Mei Terumi")
M024_T011 = Node("Missao-Time", id_missao="M024", id_time="T011", ranking="C", nome_time="Time Mei Terumi")
M027_T011 = Node("Missao-Time", id_missao="M027", id_time="T011", ranking="D", nome_time="Time Mei Terumi")

M005_T012 = Node("Missao-Time", id_missao="M005", id_time="T012", ranking="S", nome_time="Time Chojuro")
M012_T012 = Node("Missao-Time", id_missao="M012", id_time="T012", ranking="A", nome_time="Time Chojuro")
M019_T012 = Node("Missao-Time", id_missao="M019", id_time="T012", ranking="B", nome_time="Time Chojuro")
M025_T012 = Node("Missao-Time", id_missao="M025", id_time="T012", ranking="C", nome_time="Time Chojuro")
M026_T012 = Node("Missao-Time", id_missao="M026", id_time="T012", ranking="D", nome_time="Time Chojuro")

M006_T013 = Node("Missao-Time", id_missao="M006", id_time="T013", ranking="S", nome_time="Time Zabuza")
M013_T013 = Node("Missao-Time", id_missao="M013", id_time="T013", ranking="A", nome_time="Time Zabuza")
M020_T013 = Node("Missao-Time", id_missao="M020", id_time="T013", ranking="B", nome_time="Time Zabuza")
M021_T013 = Node("Missao-Time", id_missao="M021", id_time="T013", ranking="C", nome_time="Time Zabuza")
M027_T013 = Node("Missao-Time", id_missao="M027", id_time="T013", ranking="D", nome_time="Time Zabuza")

M007_T014 = Node("Missao-Time", id_missao="M007", id_time="T014", ranking="S", nome_time="Time Espadachins da Névoa")
M014_T014 = Node("Missao-Time", id_missao="M014", id_time="T014", ranking="A", nome_time="Time Espadachins da Névoa")
M017_T014 = Node("Missao-Time", id_missao="M017", id_time="T014", ranking="B", nome_time="Time Espadachins da Névoa")
M022_T014 = Node("Missao-Time", id_missao="M022", id_time="T014", ranking="C", nome_time="Time Espadachins da Névoa")
M028_T014 = Node("Missao-Time", id_missao="M028", id_time="T014", ranking="D", nome_time="Time Espadachins da Névoa")

M008_T015 = Node("Missao-Time", id_missao="M008", id_time="T015", ranking="S", nome_time="Time da Névoa Elite")
M015_T015 = Node("Missao-Time", id_missao="M015", id_time="T015", ranking="A", nome_time="Time da Névoa Elite")
M018_T015 = Node("Missao-Time", id_missao="M018", id_time="T015", ranking="B", nome_time="Time da Névoa Elite")
M023_T015 = Node("Missao-Time", id_missao="M023", id_time="T015", ranking="C", nome_time="Time da Névoa Elite")
M029_T015 = Node("Missao-Time", id_missao="M029", id_time="T015", ranking="D", nome_time="Time da Névoa Elite")


ItachiUchiha = Node("Ninja", id_ninja="N001" , id_vila= "001", nome_ninja="Itachi Uchiha")
SasukeUchiha = Node("Ninja", id_ninja="N002", id_vila="001", nome_ninja="Sasuke Uchiha")
ShisuiUchiha = Node("Ninja", id_ninja="N003", id_vila="001", nome_ninja="Shisui Uchiha")
FugakuUchiha = Node("Ninja", id_ninja="N004", id_vila="001", nome_ninja="Fugaku Uchiha")
HinataHyuga = Node("Ninja", id_ninja="N005", id_vila="001", nome_ninja="Hinata Hyuga")
NejiHyuga = Node("Ninja", id_ninja="N006", id_vila="001", nome_ninja="Neji Hyuga")
HiashiHyuga = Node("Ninja", id_ninja="N007", id_vila="001", nome_ninja="Hiashi Hyuga")
HashiramaSenju = Node("Ninja", id_ninja="N008", id_vila="001", nome_ninja="Hashirama Senju")
TobiramaSenju = Node("Ninja", id_ninja="N009", id_vila="001", nome_ninja="Tobirama Senju")
TsunadeSenju = Node("Ninja", id_ninja="N010", id_vila="001", nome_ninja="Tsunade Senju")
ShikamaruNara = Node("Ninja", id_ninja="N011", id_vila="001", nome_ninja="Shikamaru Nara")
ShikakuNara = Node("Ninja", id_ninja="N012", id_vila="001", nome_ninja="Shikaku Nara")
ChojiAkimichi = Node("Ninja", id_ninja="N013", id_vila="001", nome_ninja="Choji Akimichi")
ChozaAkimichi = Node("Ninja", id_ninja="N014", id_vila="001", nome_ninja="Choza Akimichi")
InoYamanaka = Node("Ninja", id_ninja="N015", id_vila="001", nome_ninja="Ino Yamanaka")
InoichiYamanaka = Node("Ninja", id_ninja="N016", id_vila="001", nome_ninja="Inoichi Yamanaka")
KibaInuzuka = Node("Ninja", id_ninja="N017", id_vila="001", nome_ninja="Kiba Inuzuka")
TsumeInuzuka = Node("Ninja", id_ninja="N018", id_vila="001", nome_ninja="Tsume Inuzuka")
ShinoAburame = Node("Ninja", id_ninja="N019", id_vila="001", nome_ninja="Shino Aburame")
ShibiAburame = Node("Ninja", id_ninja="N020", id_vila="001", nome_ninja="Shibi Aburame")
HiruzenSarutobi = Node("Ninja", id_ninja="N021", id_vila="001", nome_ninja="Hiruzen Sarutobi")
AsumaSarutobi = Node("Ninja", id_ninja="N022", id_vila="001", nome_ninja="Asuma Sarutobi")
KonohamaruSarutobi = Node("Ninja", id_ninja="N023", id_vila="001", nome_ninja="Konohamaru Sarutobi")
NawakiSenju = Node("Ninja", id_ninja="N024", id_vila="001", nome_ninja="Nawaki Senju")
ArataSenju = Node("Ninja", id_ninja="N025", id_vila="001", nome_ninja="Arata Senju")
Gaara = Node("Ninja", id_ninja="N026", id_vila="002", nome_ninja="Gaara")
Temari = Node("Ninja", id_ninja="N027", id_vila="002", nome_ninja="Temari")
Kankuro = Node("Ninja", id_ninja="N028", id_vila="002", nome_ninja="Kankuro")
Baki = Node("Ninja", id_ninja="N029", id_vila="002", nome_ninja="Baki")
Yashamaru = Node("Ninja", id_ninja="N030", id_vila="002", nome_ninja="Yashamaru")
Maki = Node("Ninja", id_ninja="N031", id_vila="002", nome_ninja="Maki")
Shira = Node("Ninja", id_ninja="N032", id_vila="002", nome_ninja="Shira")
SunaShinobiA = Node("Ninja", id_ninja="N033", id_vila="002", nome_ninja="Suna Shinobi A")
SunaShinobiB = Node("Ninja", id_ninja="N034", id_vila="002", nome_ninja="Suna Shinobi B")
SunaMedico = Node("Ninja", id_ninja="N035", id_vila="002", nome_ninja="Suna Médico")
MeiTerumi = Node("Ninja", id_ninja="N036", id_vila="003", nome_ninja="Mei Terumi")
ZabuzaMomochi = Node("Ninja", id_ninja="N037", id_vila="003", nome_ninja="Zabuza Momochi")
Haku = Node("Ninja", id_ninja="N038", id_vila="003", nome_ninja="Haku")
Chojuro = Node("Ninja", id_ninja="N039", id_vila="003", nome_ninja="Chojuro")
Ao = Node("Ninja", id_ninja="N040", id_vila="003", nome_ninja="Ao")
SuigetsuHozuki = Node("Ninja", id_ninja="N041", id_vila="003", nome_ninja="Suigetsu Hozuki")
MangetsuHozuki = Node("Ninja", id_ninja="N042", id_vila="003", nome_ninja="Mangetsu Hozuki")
KisameHoshigaki = Node("Ninja", id_ninja="N043", id_vila="003", nome_ninja="Kisame Hoshigaki")

R1 = Relationship(ItachiUchiha, "PARTICIPA DE", M001_T001)
R2 = Relationship(SasukeUchiha, "PARTICIPA DE", M001_T001)
R3 = Relationship(ShikamaruNara, "PARTICIPA DE", M001_T001)
R4 = Relationship(ChojiAkimichi, "PARTICIPA DE", M001_T001)
R5 = Relationship(InoYamanaka, "PARTICIPA DE", M001_T001)
R6 = Relationship(ShisuiUchiha, "PARTICIPA DE", M003_T002)
R7 = Relationship(FugakuUchiha, "PARTICIPA DE", M003_T002)
R8 = Relationship(ShikakuNara, "PARTICIPA DE", M003_T002)
R9 = Relationship(ChozaAkimichi, "PARTICIPA DE", M003_T002)
R10 = Relationship(KibaInuzuka, "PARTICIPA DE", M003_T002)
R11 = Relationship(HinataHyuga, "PARTICIPA DE", M005_T003)
R12 = Relationship(NejiHyuga, "PARTICIPA DE", M006_T003)
R13 = Relationship(HiashiHyuga, "PARTICIPA DE", M013_T003)
R14 = Relationship(KonohamaruSarutobi, "PARTICIPA DE", M019_T003)
R15 = Relationship(NawakiSenju, "PARTICIPA DE", M023_T003)
R16 = Relationship(HashiramaSenju, "PARTICIPA DE", M007_T004)
R17 = Relationship(TobiramaSenju, "PARTICIPA DE", M007_T004)
R18 = Relationship(TsumeInuzuka, "PARTICIPA DE", M007_T004)
R19 = Relationship(ShibiAburame, "PARTICIPA DE", M007_T004)
R20 = Relationship(ArataSenju, "PARTICIPA DE", M007_T004)
R21 = Relationship(TsunadeSenju, "PARTICIPA DE", M008_T005)
R22 = Relationship(InoichiYamanaka, "PARTICIPA DE", M008_T005)
R23 = Relationship(ShinoAburame, "PARTICIPA DE", M008_T005)
R24 = Relationship(HiruzenSarutobi, "PARTICIPA DE", M008_T005)
R25 = Relationship(AsumaSarutobi, "PARTICIPA DE", M008_T005)
R26 = Relationship(Gaara, "PARTICIPA DE", M009_T006)
R27 = Relationship(Temari, "PARTICIPA DE", M009_T006)
R28 = Relationship(Kankuro, "PARTICIPA DE", M010_T007)
R29 = Relationship(Baki, "PARTICIPA DE", M010_T007)
R30 = Relationship(Yashamaru, "PARTICIPA DE", M001_T008)
R31 = Relationship(Shira, "PARTICIPA DE", M001_T008)
R32 = Relationship(Maki, "PARTICIPA DE", M002_T009)
R33 = Relationship(SunaShinobiA, "PARTICIPA DE", M002_T009)
R34 = Relationship(SunaShinobiB, "PARTICIPA DE", M003_T010)
R35 = Relationship(SunaMedico, "PARTICIPA DE", M003_T010)
R36 = Relationship(MeiTerumi, "PARTICIPA DE", M004_T011)
R37 = Relationship(Chojuro, "PARTICIPA DE", M004_T011)
R38 = Relationship(ZabuzaMomochi, "PARTICIPA DE", M005_T012)
R39 = Relationship(Haku, "PARTICIPA DE", M005_T012)
R40 = Relationship(Ao, "PARTICIPA DE", M006_T013)
R41 = Relationship(SuigetsuHozuki, "PARTICIPA DE", M006_T013)
R42 = Relationship(MangetsuHozuki, "PARTICIPA DE", M007_T014)
R43 = Relationship(KisameHoshigaki, "PARTICIPA DE", M008_T015)

graph.create(ItachiUchiha | SasukeUchiha | ShisuiUchiha | FugakuUchiha | HinataHyuga | NejiHyuga |
             HiashiHyuga | HashiramaSenju | TobiramaSenju | TsunadeSenju | ShikamaruNara | ShikakuNara |
             ChojiAkimichi | ChozaAkimichi | InoYamanaka | InoichiYamanaka | KibaInuzuka | TsumeInuzuka |
             ShinoAburame | ShibiAburame | HiruzenSarutobi | AsumaSarutobi | KonohamaruSarutobi | NawakiSenju |
             ArataSenju | Gaara | Temari | Kankuro | Baki | Yashamaru | Maki | Shira | SunaShinobiA | SunaShinobiB |
             SunaMedico | MeiTerumi | ZabuzaMomochi | Haku | Chojuro | Ao | SuigetsuHozuki | MangetsuHozuki | KisameHoshigaki)

graph.create(M001_T001 | M002_T001 | M011_T001 | M016_T001 | M017_T001 | M021_T001 | M026_T001 |
             M003_T002 | M004_T002 | M012_T002 | M018_T002 | M022_T002 | M027_T002 | M005_T003 |
             M006_T003 | M013_T003 | M019_T003 | M023_T003 | M028_T003 | M007_T004 | M014_T004 |
             M016_T004 | M020_T004 | M024_T004 | M026_T004 | M029_T004 | M008_T005 | M015_T005 |
             M017_T005 | M021_T005 | M025_T005 | M030_T005 | M009_T006 | M011_T006 | M018_T006 |
             M022_T006 | M026_T006 | M010_T007 | M012_T007 | M019_T007 | M023_T007 | M027_T007 |
             M001_T008 | M013_T008 | M020_T008 | M021_T008 | M028_T008 | M002_T009 | M014_T009 |
             M016_T009 | M022_T009 | M029_T009 | M003_T010 | M015_T010 | M017_T010 | M023_T010 |
             M030_T010 | M004_T011 | M011_T011 | M018_T011 | M024_T011 | M027_T011 | M005_T012 |
             M012_T012 | M019_T012 | M025_T012 | M026_T012 | M006_T013 | M013_T013 | M020_T013 |
             M021_T013 | M027_T013 | M007_T014 | M014_T014 | M017_T014 | M022_T014 | M028_T014 |
             M008_T015 | M015_T015 | M018_T015 | M023_T015 | M029_T015)

graph.create(R1 | R2 | R3 | R4 | R5 | R6 | R7 | R8 | R9 | R10 | R11 | R12 |
             R13 | R14 | R15 | R16 | R17 | R18 | R19 | R20 | R21 | R22 | R23 | R24 | R25
             | R26 | R27 | R28 | R29 | R30 | R31 | R32 | R33 | R34 | R35 | R36 | R37 | R38 | R39 | R40 | R41 | R42 | R43)


print("Nós e relacionamentos criados com sucesso!")


Nós e relacionamentos criados com sucesso!
